In [208]:
from numpy import random,exp,array, dot
import numpy as np
import pandas as pd
import requests
import io
import copy

In [ ]:
url="intrusion_detection/data.csv"
df=pd.read_csv(url)

In [ ]:
print(df)

In [ ]:
epoch_error=[]
output1=0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import mixture
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:

class NeuralNetwork():
    def ini(self, gateInput, gateOutput):
        self.gateInput = gateInput
        self.input_shape = (1,29)
        print(self.input_shape)
        self.layer_nodes = 14
        self.hidden_layers = 1
        self.a = {}
        self.gateOutput = gateOutput
        self.output_shape = (1,29)
        print((self.output_shape))

    def softmax(self,a):
        mx = np.sum(a,axis = 1,keepdims = True)
        mx1=mx
        ex1 = np.divide(a,mx1)
        return ex1
    
    def initialization(self,):
        self.weights = {}   
        input1=self.input_shape[1]
        layernodes=self.layer_nodes
        self.weights[0] = (2 * np.random.random((input1,layernodes)) - 1)*0.01
        for i in range(1,self.hidden_layers):
            hiddennodes=self.layer_nodes
            self.weights[i] = (2 * np.random.random((hiddennodes, hiddennodes)) - 1)*0.01
        output1=self.output_shape[1]
        self.weights[self.hidden_layers] = (2 * np.random.random((layernodes,output1)) - 1)*0.01

    def sigmoid(self, x):
        return x
    
    def error(self,y1,y_hat):
        temp=(y1-y_hat)
        y_minus_yhat_square=np.sum(temp*temp,axis = 1)
        error1=0.5*y_minus_yhat_square
        return np.mean(error1)
    
    def think(self, x):
        input1=x
        weight0=self.weights[0]
        z=dot(x,weight0)
        layer1 = self.sigmoid(z)
        return layer1

    def sigmoid_derivative(self, x):
        temp=x.shape
        w=np.ones(temp)
        return w
    
    

   
    
    
    def train(self, num_steps):
        for x in range(num_steps):
            #FORWARD PROP###
                initialInput=self.gateInput
                weight0=self.weights[0]
                z0=dot(initialInput,weight0)
                self.a[0] = self.sigmoid(z0)
                learning_rate=.0000001
                
                for i in range(1,self.hidden_layers):
                    current_layer_z=self.a[i-1]
                    weight_next= self.weights[i]
                    z_new=dot(current_layer_z,weight_next)
                    self.a[i] = self.sigmoid(z_new)
                
                last_hidden_layer=self.a[self.hidden_layers-1]
                last_weights=self.weights[self.hidden_layers]
                output_z=dot(last_hidden_layer,last_weights)
                output = self.sigmoid(output_z)
                
                ##END FORWARD###
                ##ERROR##
                outputError = -(self.gateOutput - self.softmax(output))
                error2=self.error(self.gateOutput,self.softmax(output))
                epoch_error.append(error2)
                output1=error2
                ##END ERROR##
                ######BACK PROPOGATION#############
                
                ##last delta
                delta = outputError * self.sigmoid_derivative(output)
                hidden_layerLength=self.hidden_layers
                out_weights_adjustment = dot(self.a[hidden_layerLength-1].T, delta)
                self.weights[hidden_layerLength] -= learning_rate*out_weights_adjustment
                for i in np.arange(hidden_layerLength-1,0,-1):
                    activation_derivative=self.sigmoid_derivative(self.a[i])
                    delta = dot(delta, self.weights[i+1].T)*activation_derivative
                    weight_2_adjustment = dot(self.a[i-1].T, delta)
                    x1=self.weights[i]-learning_rate*weight_2_adjustment
                    self.weights[i] = x1
                activation_derivative=self.sigmoid_derivative(self.a[0])
                delta = dot(delta, self.weights[1].T) * activation_derivative
                weight_1_adjustment = dot(self.gateInput.T, delta)
                self.weights[0] -=learning_rate*weight_1_adjustment



In [ ]:
file = df
X= file.drop(['xAttack'],axis=1)
X=(X-X.mean())/X.std()

In [ ]:

neural_network = NeuralNetwork()
neural_network.ini(X.iloc[0*10000:0*10000+10000],X.iloc[0*10000:0*10000+10000])
neural_network.initialization()
neural_network.train(50)
for i in range(1,11):
    print(i)
    neural_network.ini(X.iloc[i*10000:i*10000+10000],X.iloc[i*10000:i*10000+10000])
    neural_network.train(50)


In [ ]:
print(epoch_error[-1])
# output1

In [ ]:
X1=neural_network.think(X)
# X1

In [ ]:
l=[];
x=len(X1);
# print(x)
for i in range(0,x):
    l.append(i)
Df = pd.DataFrame(data = X1,
                  index=l)
data=Df.values
k=5
c = data.shape[1]
# Df

Df=pd.concat([Df,file['xAttack']],axis=1)
Df.to_csv("intrusion_detection/newDataLinear.csv",encoding='utf-8',index=False)

In [ ]:
df=pd.read_csv("intrusion_detection/newDataLinear.csv")
X2= df.drop(['xAttack'],axis=1)
y =pd.DataFrame(df['xAttack'])
# X=(X-X.mean())/X.std()
labelencoder=LabelEncoder()
y=labelencoder.fit_transform(y)
y=pd.DataFrame(data=y)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2, random_state = 3)

In [ ]:
print(type(X_train))
print(type(y))

In [ ]:
data=X_train.values
X_final=pd.concat([X_train,y_train],axis=1)
print(type(X_final))

In [ ]:
data1=X_final.values
data2=X_test.values
# y_train=pd.concat([y_train,pd.get_dummies(y_train['xAttack'],prefix='xAttack')],axis=1).drop(['xAttack'],axis=1)
# y=pd.concat([y,pd.get_dummies(y['label'],prefix='label')],axis=1).drop(['label'],axis=1)

## Kmeans##

In [ ]:
mean = np.mean(data,axis = 0)
std = np.std(data, axis = 0)
k=5
c = data.shape[1]
print(c)

def euclidean_distance(x, y): 
        return np.sqrt(np.sum((x - y) ** 2))


In [ ]:
centers = np.random.randn(k,c)*std+mean
n=data.shape[0]

centers_old = np.zeros(centers.shape)
centers_new = copy.deepcopy(centers)
error=0
for i in range(k):
    temp4=euclidean_distance(centers_new[i],centers_old[i])
    error = error+temp4
clusters = np.zeros(n)
distances = np.zeros((n,k))
itr=0
ans=[]
ans1=[]
while error!=0:
    for i in range(k):
        #getting distance of all point wrt to each cluster center
        distances[:,i] = np.linalg.norm(data - centers_new[i], axis=1)
    #assigning the cluster to each point whose distance is minimum
    clusters = np.argmin(distances, axis = 1)
    centers_old = copy.deepcopy(centers_new)
    for i in range(k):
        #if cluster has no point than reinitialize the process
        if(len(data[clusters==i])==0):
            clusters = np.zeros(n)
            centers = np.random.randn(5,14)
            temp=0
            for j in range(k):
                centers[j]=centers[j]*std+mean
            centers_old = np.zeros(centers.shape)
            distances = np.zeros((n,k))
            centers_new = copy.deepcopy(centers)
            for i in range(k):
                temp2=euclidean_distance(centers_new[i],centers_old[i])
                temp = temp+temp2
            error=temp
            itr=0
            break;
        else:
            #assigning new center base on assignment of points i.e mean of all the points that belog to cluster==i
            centers_new[i] = np.mean(data[clusters == i], axis=0)
    temp=0
    for i in range(k):
        temp3=euclidean_distance(centers_new[i],centers_old[i])
        temp = temp+temp3
    error=temp
    itr=itr+1

for i in range(k):
    ans.append(np.count_nonzero(clusters==i))
    ans1.append(np.count_nonzero(y==i))
# print(ans)
# print(ans1)

def purity_score_Kmeans(y_true, y_pred):
    contingency_matrix =metrics.cluster.contingency_matrix(y_true, y_pred)
    num2=np.sum(np.amax(contingency_matrix, axis=0))
    print(np.sum(np.amax(contingency_matrix, axis=0)))
    print(np.sum(contingency_matrix))
    deno=np.sum(contingency_matrix)
    return num2/deno 


print("--------------Purity of Kmeans----------------------")
print(purity_score_Kmeans(y_train,clusters))

## GMM ##

In [ ]:
gm=mixture.GaussianMixture(n_components=5).fit(data2)
labels = gm.predict(data2)

def purity_score_GMM(y_true, y_pred):
    contingency_matrix =metrics.cluster.contingency_matrix(y_true, y_pred)
    num2=np.sum(np.amax(contingency_matrix, axis=0))
    print(np.sum(np.amax(contingency_matrix, axis=0)))
    print(np.sum(contingency_matrix))
    deno=np.sum(contingency_matrix)
    return num2/deno 


print("----------Gaussian Purity Score------------------")
print(purity_score_GMM(y_test,labels))

# contingency_matrix =metrics.cluster.contingency_matrix(y_test, labels)
# ans = 0
# result=0
# cont = contingency_matrix
# for i in cont:
# #     temp=max(i,axis=0)
# #     ans+=temp
#     max2=0
#     for k in i:
#         result=result+k
#         if(k>max2):
#             max2=k
#     ans=ans+max2
        

## Hierarchical Clustering ##

In [ ]:
data3=data[0:20000]
y_test1=y_train[0:20000]
from sklearn.cluster import AgglomerativeClustering
k=5


In [ ]:
# from sklearn.metrics.cluster import contingency_matrix
def purity_score_HRR(y_true, y_pred):
    contingency_matrix =metrics.cluster.contingency_matrix(y_true, y_pred)
    num2=np.sum(np.amax(contingency_matrix, axis=0))
    print(np.sum(np.amax(contingency_matrix, axis=0)))
    print(np.sum(contingency_matrix))
    deno=np.sum(contingency_matrix)
    return float(num2)/float(deno) 
# ans = 0
# cont = contingency_matrix(labels,y_test1)
# for i in cont:
#     temp=max(i)
#     ans+=temp
# num=ans
cluster = AgglomerativeClustering(n_clusters=k, affinity='euclidean', linkage='single')  
labels = cluster.fit_predict(data3)  
# deno_len=len(data3)
# purity = ans/len(data3)
print("--------------------Purity Score  -----------------")
print(purity_score_HRR(y_test1,labels))

## PIE CHART##

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
sizes = [85.94, 84.19, 53.9]
colors = ['gold', 'blue', 'yellow']
labels =  'KMeans', 'GMM', 'Hierarchical',
explode = (0, 0, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()